In [1]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI
import torch
from sentence_transformers import SentenceTransformer

/home/patrick/rag_from_scratch/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [4]:
# Load the pre-trained model
embedder = SentenceTransformer("all-MiniLM-L6-v2") # Fazer o embedding usando esse

In [5]:
pdf_path = "/home/patrick/rag_from_scratch/AI_Information.pdf"

In [7]:
def extract_text_from_pdf(pdf_path):
    mypdf = fitz.open(pdf_path)
    all_text = "" # Initizalizing an empty string to store the extarcted text

    # Iterating through each page in the pdf
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")
        all_text += text

    return all_text

In [8]:
extracted_text = extract_text_from_pdf(pdf_path)

In [24]:
def chunk_text(text, n, overlap):
    chunks = []

    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i+n])

    return chunks

In [25]:
text_chunks = chunk_text(extracted_text, 1000, 100)

In [26]:

print("Number of text chunks:", len(text_chunks))

print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 38

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

In [27]:
def create_embeddings(text):
    response = embedder.encode(text)
    return response

response = create_embeddings(text_chunks)

In [28]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [47]:
def semantic_search(query, text_chunks, embeddings, k=5, context_size=2):
    query_embedding = create_embeddings(query)
    similarity_scores = []

    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_index = similarity_scores[0][0]
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # Return the relevant chunk along with its neighboring context chunks
    return [text_chunks[i] for i in range(start, end)]

In [48]:
with open('val.json') as f:
    data = json.load(f)
query = data[0]['question']

top_chunks = semantic_search(query, text_chunks, response, k=2)

print("Query:", query)

for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:

privacy and human rights. 
Public Perception and Trust 
Public perception and trust in AI are essential for its widespread adoption and positive social 
impact. Building trust requires transparency, explainability, and responsible development and 
deployment of AI systems. 
Global Collaboration 
Addressing the social impact of AI requires global collaboration and cooperation. This includes 
sharing knowledge, developing standards, and promoting responsible AI practices across 
borders. 
Chapter 14: AI and Smart Cities 
Urban Planning and Management 
AI enhances urban planning and management by analyzing data, optimizing resource allocation, 
and improving city services. AI-powered systems support sustainable urban development, 
enhance quality of life, and promote efficient city operations. 
Smart Transportation 
AI-powered smart transportation systems optimize traffic flow, reduce congestion, and enhance 


In [49]:
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="gpt-3.5-turbo-1106"):
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

ai_response = generate_response(system_prompt, user_prompt)

In [50]:
ai_response.choices[0].message.content

'Explainable AI (XAI) aims to make AI systems more transparent and understandable. Research in XAI focuses on developing methods for explaining AI decisions, enhancing trust, and improving accountability. It is considered important because it contributes to building trust in AI systems, which is essential for their widespread adoption and positive social impact.'

In [51]:
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

print(evaluation_response.choices[0].message.content)

The AI response is very close to the true response, capturing the essence of Explainable AI (XAI) and its importance in building trust, enhancing accountability, and ensuring fairness in AI systems. Therefore, the score is 1.


In [52]:
import json
import asyncio
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, SimilarityFunction

# Initialize the SentenceTransformer model and set the similarity function
model = SentenceTransformer("all-MiniLM-L6-v2")
model.similarity_fn_name = SimilarityFunction.DOT

In [53]:
async def process_validation_data(k):
    system_prompt = """You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly and exactly from the provided context, respond with: 'I do not have enough information to answer that.'
    First think about the keywords from the question and then use them to elaborate the answer.
    The response needs to be just the answer sentence
    
    """
    # Load the validation data from the JSON file
    with open('val.json') as f:
        data = json.load(f)

    # List to store the results for each sample
    results = []

    # Iterate over each example in the validation data
    for idx, item in enumerate(data):
        query = item['question']
        ideal_answer = item['ideal_answer']
        
        # Retrieve the top k most relevant context chunks
        top_chunks = semantic_search(query, text_chunks, response, k=k)
        
        # Create the user prompt by combining all context chunks and the query
        context_prompt = "\n".join([
            f"Context {i + 1}:\n{chunk}\n=====================================\n"
            for i, chunk in enumerate(top_chunks)
        ])
        user_prompt = f"{context_prompt}\nQuestion: {query}"
        
        # Generate the AI response using the system prompt and the user prompt
        ai_response = generate_response(system_prompt, user_prompt).choices[0].message.content
        
        # Evaluate similarity using SentenceTransformer
        # Encode the AI response and ideal answer
        embedding_response = model.encode([ai_response])
        embedding_ideal = model.encode([ideal_answer])
        # Compute similarity score (result is a 1x1 matrix; extract the single value)
        similarity_matrix = model.similarity(embedding_response, embedding_ideal)
        score = similarity_matrix[0][0].numpy()
        
        # Prepare the result dictionary with dynamic context columns
        result = {
            "Query": query,
            "Ideal Answer": ideal_answer,
            "AI Response": ai_response,
            "Score": score
        }
        # Add each context as its own column
        for i, chunk in enumerate(top_chunks):
            result[f"Context {i + 1}"] = chunk
        
        # Append the result to the list
        results.append(result)

    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    return df

In [54]:
import nest_asyncio
nest_asyncio.apply()

In [55]:
for k in range(1, 9):
    print(asyncio.run(process_validation_data(k=k))['Score'].mean())

0.6662616729736328
0.6569515228271484
0.6584525108337402
0.6584525108337402
0.6584525108337402
0.6584525108337402
0.6584525108337402
0.6662616729736328
